# Rede Neural

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'
CONFIG_PATH = MODEL_PATH+'/my_ssd_mobnet/pipeline.config'
CHECKPOINT_PATH = MODEL_PATH+'/my_ssd_mobnet/'

import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/pipeline.config'

config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# DEFINI VARIÁVEIS

In [ ]:
def inicia_variaveis():
    
    global tempoInicialGeral, tempoFinalGeral, identificadas, baixo, alto, fator_imgs, altura_minima_ponto, altura_maxima_ponto
    global corretas, erradas, info, sensibilidade, mediaHistograma, LER_IMG, ESCREVE_IMG, distancia_pontos_raio_px, delta_distancia_lidar, reducao_caixa_fruta
    
    global TIPO_BASE, limiarLarguraPedunculo, limiarAlturaPedunculo, NOME_IMAGEM, NOME_JSON, NOME_TXT, METODO, min_samples,metodo_pontos_curva,altura_caixa_pedunculo
    
    tempoInicialGeral = 0
    tempoFinalGeral = 0
    identificadas = 0
    
    corretas =        []
    erradas =         []
    info =            []
    sensibilidade =   []
    mediaHistograma = []

    limiarAlturaPedunculo  = 0.3
    limiarLarguraPedunculo = 0.2

    baixo = [2, 0, 0]
    alto  = [25, 255, 255]

    distancia_pontos_raio_px = 2

    fator_imgs = 7.5
    
    altura_minima_ponto = 3
    altura_maxima_ponto = 8
    
    delta_distancia_lidar = 10
    
    reducao_caixa_fruta = 0.2
    
    altura_caixa_pedunculo = 10
    
    min_samples = 4
    metodo_pontos_curva = "IsolationForest"
    #metodo_pontos_curva = "Outliners"
    
    TIPO_BASE = "3D"
    
    METODO = "DBSCAN"
    #METODO = "Kmeans"
    #METODO = "Heuristica"
    
    #NOME_IMAGEM = "_MANGO_LEAFLESS"
    #NOME_JSON   = "_MANGO_MAP_JSON"
    
    #PASTA = "VALID_CUT_POINT_2D/"
    #LER_IMG = "VI2D_MANGO_LEAFLESS/"
    
    NOME_IMAGEM = "_MANGO_IMAGE"
    NOME_JSON   = "_MANGO_MAP_JSON"
    
    PASTA = "VALID_LIDAR_IMAGES_3D/"
    LER_IMG = "VALIDACAO_SVCF/"
    
    LER_IMG = PASTA + LER_IMG
    ESCREVE_IMG = "RESULTADO_IMAGENS_VALIDACAO_3D/"
    
    NOME_TXT = LER_IMG[:-1] + "_" + TIPO_BASE + "_" + METODO

# FUNÇÃO PRINCIPAL

In [ ]:
import cv2 
import numpy as np
import time
import traceback
import math
import json
import os

import PC_SVCF as pc
import Lidar as lidar
import Valida_SVCF as vl

inicia_variaveis()

print("Executando...")
print("Log..." + NOME_TXT)
arquivo = open(NOME_TXT + ".csv", "w")

if(TIPO_BASE == "2D"):
    
    arquivo.write("IMAGEM,ID_MANGA,SCORES,HUE,%_MANGA,%_PONTO,MEDIA\n")
    
    tamanhoLetraTexto = 0.6
    fatorLetra = 20
    
else:
    
    arquivo.write("IMAGEM,ID_MANGA,SCORES,FATOR_CM_PX,DISTÂNCIA_MANGA,HUE,DISTÂNCIA_PONTO,%_MANGA,%_PEDÚNCULO,DISTÂNCIA HORI,%_PONTO,MEDIA\n")
    
    tamanhoLetraTexto = 1.0
    fatorLetra = 35

tempoInicialGeral = time.time()

id_imagem = 1
qtdImage  = 72

pontosJson = ""
fator_cm_px = ""

while(id_imagem <= qtdImage):
    
    inicia_variaveis()

    print("############## CONFIGURAÇÕES DE LEITURA DOS ARQUIVOS ######################\n")
    
    caminho_imagem = LER_IMG + str(id_imagem) + NOME_IMAGEM +  ".jpg"
    print(f"Caminho da imagem: {caminho_imagem}")
    
    imagem_original = cv2.imread(LER_IMG + str(id_imagem) + NOME_IMAGEM + ".jpg")
        
    caminho_json = LER_IMG + str(id_imagem) + NOME_JSON + ".json"
    print(f"Caminho do json:   {caminho_json}\n")
    
    if(TIPO_BASE == "3D"):

        pontosJson = lidar.organizaJSON(caminho_json)
        
    #Copia a imagem original
    imagem_original_copia = np.array(imagem_original)
    
    imagem_detectada = imagem_original_copia.copy() #USADA DURANTE A EXECUÇÃO DO CÓDIGO
    imagem_pedunculo = imagem_original_copia.copy() #USADA PARA A ÁREA DO PEDÚNCULO
    
    #Copia a imagem na versão reduzida
    imagem_original_reduzida_aux = cv2.resize(imagem_original, (192, 256))
    imagem_original_reduzida = np.array(imagem_original_reduzida_aux.astype(np.uint8))
    
    ################################ CONFIGURAÇÕES DA REDE NEURAL ################################
    
    category_index = label_map_util.create_category_index_from_labelmap(ANNOTATION_PATH+'/label_map.pbtxt')
        
    input_tensor = tf.convert_to_tensor(np.expand_dims(imagem_original_copia, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor) 

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    
    detection_threshold = 0.8

    viz_utils.visualize_boxes_and_labels_on_image_array(
                imagem_detectada,
                detections['detection_boxes'],
                detections['detection_classes'] + label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                skip_scores= True,
                skip_labels= True,
                skip_boxes= False,
                agnostic_mode=False)
  
    scores = list(filter(lambda x: x> detection_threshold, 
    detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]
        
    width =  imagem_detectada.shape[1]
    height = imagem_detectada.shape[0]
        
    sensibilidade.append(scores)
    
    quantidade_fruta_identificadas = len(scores)
    
    ################################ INICIO DO PROCESSO DE LOCALIZAÇÃO DO PONTO DE CORTE ################################
    
    if(quantidade_fruta_identificadas > 0):
                
        print(f"Imagem: {id_imagem} - Quantidade de fruta identificada: {quantidade_fruta_identificadas}\n")
                
        try:
            
            posicaoTextoY = fatorLetra
            
            for id_manga_localizada, box in enumerate(boxes):
                
                arquivo.write(f"{str(id_imagem)},")
                                
                print(f"Manga localizada: {id_manga_localizada}")
                
                if(id_manga_localizada < 2):
                    
                    arquivo.write(f"{str(id_manga_localizada)},")
                    arquivo.write(f"{str(round(scores[id_manga_localizada], 4))},")
                                                            
                    ################################# ÁREA DA MANGA #################################

                    roi = box * [height, width, height, width]
                    
                    xt_ia_manga = roi[1]
                    yt_ia_manga = roi[0]
                    
                    xb_ia_manga = roi[3]
                    yb_ia_manga = roi[2]

                    xt_ia_manga = int(xt_ia_manga)
                    xb_ia_manga = int(xb_ia_manga)
                    yt_ia_manga = int(yt_ia_manga)
                    yb_ia_manga = int(yb_ia_manga)
                    
                    cv2.putText(imagem_detectada, str(id_manga_localizada), (xt_ia_manga, yt_ia_manga - 5), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)
                                        
                    centroCaixaFrutaX = int((xt_ia_manga + xb_ia_manga) / 2)
                    centroCaixaFrutaY = int((yt_ia_manga + yb_ia_manga) / 2)
                    
                    centroCaixaFrutaXLidar = int(centroCaixaFrutaX / fator_imgs)
                    centroCaixaFrutaYLidar = int(centroCaixaFrutaY / fator_imgs)
                    
                    media_imagem = [ ]
                    
                    if(TIPO_BASE == "3D"):
                    
                        #Função para encontrar o fator cm_px referente a manga
                        fator_cm_px = pc.obter_cm_px(4, 2, centroCaixaFrutaXLidar, centroCaixaFrutaYLidar, pontosJson)
                                                
                        arquivo.write(f"{str(round(fator_cm_px, 4))},")

                        print(f"Fator cm_px: {fator_cm_px:.5f}")
                                            
                        altura_caixa_pedunculo = 2.5 / fator_cm_px * fator_imgs

                        distanciaFruta = lidar.measureDistanceOnePoint((centroCaixaFrutaXLidar, centroCaixaFrutaYLidar), pontosJson)

                        print(f"Distancia Manga: {distanciaFruta:.2f} cm")
                        
                        arquivo.write(f"{str(round(distanciaFruta, 2))},")

                        
                    #Faz um corte na área da manga
                    areaManga = imagem_detectada[yt_ia_manga:yb_ia_manga, xt_ia_manga:xb_ia_manga]
                    
                    #Circulo no centro da área da manga
                    cv2.circle(imagem_detectada, (int(centroCaixaFrutaX), int(centroCaixaFrutaY)), 5, (255, 0, 90), -1)
                    
                    #Desenha um retangulo na área da manga
                    cv2.rectangle(imagem_detectada, (xt_ia_manga, yt_ia_manga), (xb_ia_manga, yb_ia_manga), (255,0,0), 2)
                    
                    
                    ################################# ÁREA DO PEDÚNCULO ################################# 

                    #Chama a função que prevê a área do pedúnculo
                    areaPedunculo = pc.preveAreaPedunculo(xt_ia_manga, yt_ia_manga, xb_ia_manga, yb_ia_manga, limiarLarguraPedunculo, limiarAlturaPedunculo, -altura_caixa_pedunculo)
                    
                    #x1                y1                 x2                 y2
                    xt_area_pedunculo, yt_area_pedunculo, xb_area_pedunculo, yb_area_pedunculo = areaPedunculo[0], areaPedunculo[1], areaPedunculo[2], areaPedunculo[3]

                    #Desenha um retangulo na área do pedúnculo        
                    cv2.rectangle(imagem_detectada, (xt_area_pedunculo, yt_area_pedunculo), (xb_area_pedunculo, yb_area_pedunculo), (0, 0, 255), 2)

                    #Coordenadas da área da manga
                    TopLeftX = xt_area_pedunculo
                    TopLeftY = yb_area_pedunculo

                    #Faz o corte na área prevista do pedúnculo
                    corteAreaPedunculo = imagem_pedunculo[yb_area_pedunculo:yt_area_pedunculo, xt_area_pedunculo:xb_area_pedunculo]                    
                    
                    if(len(corteAreaPedunculo) < 1):
                        
                        print("Área do pedúnculo muito pequena")
                        
                    else:
                    
                        #Circulo no centro do topo da área da fruta
                        cv2.circle(imagem_detectada, (int(centroCaixaFrutaX), int(yt_ia_manga)), 5, (0, 0, 255), -1)

                        if(TIPO_BASE == "3D"):

                            #Distancia do centro do topo da caixa do fruta
                            distanciaTopoCaixaFruto = lidar.measureDistanceOnePoint((centroCaixaFrutaXLidar, int(yt_ia_manga / fator_imgs)), pontosJson)

                            print(f"Distancia Topo Caixa: {distanciaTopoCaixaFruto:.2f} cm")

                        #Função que encontra o ponto de corte final de acordo com os parâmetros
                        ponto_final = pc.localiza_ponto_final(str(id_imagem),
                                                str(id_manga_localizada),
                                                corteAreaPedunculo, 
                                                baixo, 
                                                alto, 
                                                METODO, 
                                                ESCREVE_IMG, 
                                                True, 
                                                "top", 
                                                7, 
                                                min_samples, 
                                                TIPO_BASE, 
                                                metodo_pontos_curva, 
                                                TopLeftX, 
                                                TopLeftY, 
                                                pontosJson, 
                                                altura_minima_ponto, 
                                                (centroCaixaFrutaX, centroCaixaFrutaY), 
                                                (centroCaixaFrutaX, yt_ia_manga))

                        pontoX, pontoY, valorHue, distanciaPonto = ponto_final[0], ponto_final[1], ponto_final[2], ponto_final[3]

                        arquivo.write(f"{str(valorHue)},")

                        cv2.circle(imagem_detectada, (pontoX, pontoY), 5, (255, 0, 255), -1)
                        cv2.line(imagem_detectada, (pontoX, pontoY), (centroCaixaFrutaX, yt_ia_manga), (255, 0, 255), 3)
                        cv2.circle(imagem_detectada, (pontoX, pontoY), 6, (255, 0, 0), 2)

                        #Função para validar o sistema
                        validacao = vl.valida_svcf(imagem_detectada, 
                                       None,
                                       id_imagem,
                                       id_manga_localizada,
                                       TIPO_BASE,
                                       (xt_ia_manga, yt_ia_manga, xb_ia_manga, yb_ia_manga),
                                       (xt_area_pedunculo, yt_area_pedunculo, xb_area_pedunculo, yb_area_pedunculo),
                                       caminho_json, 
                                       fator_cm_px,
                                       (pontoX, pontoY),
                                       altura_minima_ponto,
                                       distanciaPonto)

                        por_manga, por_pedunculo, por_ponto, distancia_hor = validacao[0], validacao[1], validacao[2], validacao[3]

                        if(TIPO_BASE == "3D"):

                            arquivo.write(f"{str(round(distanciaPonto, 2))},")

                        arquivo.write(f"{str(por_manga)},")

                        print(f"Porcentagem Manga:     {por_manga} %")
                        cv2.putText(imagem_detectada, "Precisao Manga - " + str(id_manga_localizada) + ": "+ str(por_manga) + " %", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                        posicaoTextoY += fatorLetra

                        if(por_pedunculo != -1):

                            print(f"Porcentagem Pedunculo: {por_pedunculo} %")
                            print(f"Distancia Horizontal:  {distancia_hor} cm")

                            arquivo.write(f"{str(por_pedunculo)},")
                            arquivo.write(f"{str(distancia_hor)},")

                            cv2.putText(imagem_detectada,"Precisao Pedunculo: " + str(por_pedunculo) + " %", (20, posicaoTextoY),cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                            posicaoTextoY += fatorLetra

                            cv2.putText(imagem_detectada, "Distancia Hor: " + str(distancia_hor) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                            posicaoTextoY += fatorLetra

                        arquivo.write(f"{str(por_ponto)},")
                        print(f"Porcentagem Ponto:     {por_ponto} %")
                        cv2.putText(imagem_detectada, "Precisao Ponto: " + str(por_ponto) + " %", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                        posicaoTextoY += fatorLetra

                        if(TIPO_BASE == "3D"):

                            media_imagem = [por_manga, por_pedunculo, por_ponto]

                            cv2.putText(imagem_detectada, "Altura Ponto: " + str(distanciaPonto) + " cm", (20, posicaoTextoY), cv2.FONT_HERSHEY_SIMPLEX, tamanhoLetraTexto, (0, 0, 255), 2, 1)

                            posicaoTextoY += fatorLetra + 15

                        else:

                            media_imagem = [por_manga, por_ponto]

                        arquivo.write(f"{str(round(np.mean(media_imagem), 2))}\n")
                        
                        #Salva imagem representativa da calibração
                        cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_a_calibracao.png", imagem_original_reduzida)
                        
                        #Salva imagem da área do pedúnculo RGB
                        cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_b_area_pedunculo.jpg", corteAreaPedunculo)

             
                cv2.imwrite(ESCREVE_IMG + str(id_imagem) + "_" + str(id_manga_localizada) + "_j_ponto_final.jpg", imagem_detectada)
                    
            print("-*-*-*-*-*-*-*-*-*-*-*")
                        
            id_imagem = id_imagem + 1
                  
        except:

            print(f"{id_imagem} - Erro desconhecido\n") 
            
            traceback.print_exc()

            id_imagem = id_imagem + 1
            
    else:
        
        print("\n{} - Fruta nao identificada\n".format(id_imagem))
        
        id_imagem = id_imagem + 1
        
tempoFinalGeral = time.time()

arquivo.close()
print("\n\nFinalizando...")